In [1]:
import geemap
import ee
from Monarch import user_gee

Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [2]:
point = Map.draw_last_feature.geometry()

In [3]:
image = images = ee.ImageCollection("LANDSAT/LT05/C01/T1").filterBounds(point)\
                    .sort("CLOUD_COVER").first()
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2']}, "imags")

In [ ]:
NDVI = image.normalizedDifference(['B5', 'B4'])
Map.addLayer(NDVI, {}, 'ndvi')

In [10]:
def clip_dow_merge(geo: ee.Geometry, image: ee.Image, outfile: str, scale: int,
                   crs='epsg:4326', sep=0.25):
    """

    Args:
        geo: ee.Geometry, 需要下载的区域矢量几何
        image: ee.Image, 单波段影像
        outfile: str, 输出文件路径和名称，不需要文件后缀，下载的影响默认后缀为tif
        scale: int, 下载时的像元大小
        crs: str, 下载影像的投影，默认为 'epsg:4326' wgs1984投影
        sep: float, 单波段10m分辨率像元的影像裁剪大小(单位：经纬度)，默认为0.25
    Returns: None

    """
    import os
    import numpy as np
    import rasterio
    from glob import glob
    from rasterio.merge import merge
    import shutil
    import geemap
    import math
    bounds = geo.bounds()
    bands = image.bandNames().size().getInfo()
    poy = np.array(bounds.coordinates().getInfo()[0])
    min_x = poy[:, 0].min()
    max_x = poy[:, 0].max()
    min_y = poy[:, 1].min()
    max_y = poy[:, 1].max()
    step = scale / 10 * sep / (int(math.sqrt(bands))+1)
    end_x = int((max_x - min_x) / step) + 10
    end_y = int((max_y - min_y) / step) + 1
    polys = []
    for i in range(end_y):
        y1 = min_y + step * i
        y2 = min_y + step * (i + 1)
        if y2 > max_y:
            y2 = max_y
        for j in range(end_x):
            x1 = min_x + step * j
            x2 = min_x + step * (j + 1)
            if x2 > max_x:
                x2 = max_x
            poly = ee.Geometry(ee.Geometry.Rectangle([x1, y1, x2, y2]), None, False)
            polys.append(poly)
    if(len(polys) > 1):
        print(f"分割成{len(polys)}份, 开始下载:")
        path = outfile+'_mk'
        if not os.path.exists(path):
            os.makedirs(path)
        for j, i in enumerate(polys):
            geemap.ee_export_image(image, path+f'/temp_{j}.tif', scale=scale, crs=crs, region=i)
        files = glob(path+"/*.tif")
        src_files_to_mosaic = []
        for tif_f in files:
            src = rasterio.open(tif_f)
            src_files_to_mosaic.append(src)
        mosaic, out_trans = merge(src_files_to_mosaic)
        out_meta = src.meta.copy()
        out_meta.update({"driver": "GTiff",
                         "height": mosaic.shape[1],
                         "width": mosaic.shape[2],
                         "transform": out_trans,
                         })
        with rasterio.open(outfile+".tif", "w", **out_meta) as dest:
            dest.write(mosaic)
        for src in src_files_to_mosaic:
            src.close()
        shutil.rmtree(path)
    else:
        geemap.ee_export_image(image, outfile+'.tif', scale=scale, crs=crs, region=geo)
    print("download successful !!!")

In [4]:
roi = Map.draw_last_feature.geometry()

In [11]:
clip_dow_merge(roi, image.clip(roi), 'H:/Monarch/image/landsat_ndvi', 30)

分割成270份, 开始下载:
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_0.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_1.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_2.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_3.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_4.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_5.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_6.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_7.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_8.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_9.tif
Gener

Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_29.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_30.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_31.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_32.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_33.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_34.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_35.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_36.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_37.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_38.tif
Generating URL ...
Please wait ...
Data downl

Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_59.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_60.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_61.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_62.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_63.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_64.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_65.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_66.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_67.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_68.tif
Generating URL ...
Please wait ...
Data downl

Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_89.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_90.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_91.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_92.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_93.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_94.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_95.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_96.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_97.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_98.tif
Generating URL ...
Please wait ...
Data downl

Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_119.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_120.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_121.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_122.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_123.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_124.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_125.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_126.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_127.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_128.tif
Generating URL ...


Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_148.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_149.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_150.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_151.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_152.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_153.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_154.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_155.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_156.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_157.tif
Generating URL ...
Please wait ...


Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_178.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_179.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_180.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_181.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_182.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_183.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_184.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_185.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_186.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_187.tif
Generating URL ...


Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_207.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_208.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_209.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_210.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_211.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_212.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_213.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_214.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_215.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_216.tif
Generating URL ...
Please wait ...


Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_237.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_238.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_239.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_240.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_241.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_242.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_243.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_244.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_245.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_246.tif
Generating URL ...


Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_266.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_267.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_268.tif
Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\image\landsat_ndvi_mk\temp_269.tif
download successful !!!


In [ ]:
image = ee.ImageCollection("MODIS/006/MOD09GQ")\
            .filterDate("2015-01-01", "2015-01-02").select('sur_refl_b01')\
            .first()

In [7]:
image.bandNames().size().getInfo()

8